## class pyspark.sql.SparkSession(sparkContext, jsparkSession=None)
用DataSet和DataFrame编写Spark程序的入口

SparkSession的功能包括：
+ 创建DataFrame
+ 以关系型数据库中表的形式生成DataFrame，之后便可以执行SQL语句，适合小数据量的操作
+ 读取.parquet格式的文件，得到DataFrame


In [1]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
import random
import os
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3.5"

# 初始化SparkSession
spark = SparkSession.builder \
    .master("spark://master:7077") \
    .appName("TopN") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext

## 创建模拟访问数据记录

In [2]:
data = ["product" + str(random.randint(1, 10)) + " url" + \
       str(random.randint(1, 100)) for index in range(1000)]
table = sc.parallelize(data)
table.cache()    # 相当于调用persist(MEMORY_ONLY)， 将RDD作为反序列化对象存储于JVM中 
tableRDD = table.map(lambda line: line.split(" "))\
            .map(lambda words: Row(product=words[0], url=words[1]))
    
#注册为临时表，即将RDD转换成DataFrame
tableSchema = spark.createDataFrame(tableRDD)
tableSchema.createOrReplaceTempView("product_url")
result = spark.sql("select * from product_url")
result.head(5)

[Row(product='product7', url='url38'),
 Row(product='product5', url='url66'),
 Row(product='product10', url='url64'),
 Row(product='product9', url='url41'),
 Row(product='product8', url='url24')]

## 统计各个产品线下各个URL的访问次数

In [3]:
accessRDD = spark.sql("select product, url ,count(*) as access from product_url group by product, url")
accessRDD.head(5)

[Row(product='product1', url='url59', access=3),
 Row(product='product3', url='url90', access=1),
 Row(product='product8', url='url21', access=1),
 Row(product='product6', url='url41', access=2),
 Row(product='product9', url='url92', access=2)]

## 分区排序取值

Spark RDD为分区排序提供了非常方便的API：repartitionAndSortWithinPartitions(numPartition, partitionFunc, ascending, keyFunc)  
依据partitioner对RDD进行分区，并且在每个结果分区中按key进行排序  
这个操作可以一边进行重分区的shuffle操作，一边进行排序。shuffle与sort两个操作同时进行，比先shuffle再sort来说，性能可能是要高的。  

In [4]:
accessPairRDD = accessRDD.rdd.map(lambda row: ((row.product, row.url, row.access), None))
def partitionFunc(key):
    return int(key[0][7]) - 1    # 这里key的数据类型为Row

def keyFunc(key):
    return key[2]

repartitionAndSortRDD = accessPairRDD.repartitionAndSortWithinPartitions(\
                        numPartitions=10, partitionFunc=partitionFunc, ascending=False, keyfunc=keyFunc)
#分区排序之后，我们需要将不同分区的前N个值汇总，通过mapPartitions实现
#mapPartitions需要一个函数f作为参数，该函数f的参数作为一个“迭代器”，通过这个“迭代器”可以遍历分区内的所有数据
N = 5
def topNFunc(iter):
    buffer = []
    for pair in iter:
        if len(buffer) >= N:
            break
            
        buffer.append(pair[0])
    
    return buffer

rows = repartitionAndSortRDD.mapPartitions(topNFunc).collect()
# 返回值是list
print(rows[:10])

[('product1', 'url44', 6), ('product1', 'url58', 4), ('product1', 'url81', 3), ('product10', 'url12', 3), ('product10', 'url54', 3), ('product2', 'url35', 6), ('product2', 'url89', 4), ('product2', 'url55', 4), ('product2', 'url84', 3), ('product2', 'url6', 3)]


接下来我们要进行**二次排序**，即：先根据product的序号进行排序，再在此基础上按URL序号排序

In [5]:
#对结果进行排序
rows_sorted = sorted(rows, key=lambda row: (row[0], row[1]))
print(rows_sorted[:10])

[('product1', 'url44', 6), ('product1', 'url58', 4), ('product1', 'url81', 3), ('product10', 'url12', 3), ('product10', 'url54', 3), ('product2', 'url35', 6), ('product2', 'url55', 4), ('product2', 'url6', 3), ('product2', 'url84', 3), ('product2', 'url89', 4)]


### sort 与 sorted 区别：
sort 是应用在 list 上的方法，sorted 可以对所有可迭代的对象进行排序操作。  
list 的 sort 方法返回的是对已经存在的列表进行操作，无返回值，而内建函数 sorted 方法返回的是一个新的 list，而不是在原来的基础上进行的操作。  
注意：在Python3中不能使用cmp函数